In [17]:
## llm 답변 실제로 받아보자

# 1. State 정의 및 그래프 초기화

import operator
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, START
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from personaldb_prompts_v3 import (combined_table_info,
                                   tourinfo_table_info,
                                   weather_table_info,
                                   accommodation_table_info,
                                   restaurant_table_info,
                                   sql_generation_prompt,
                                   tourinfo_query_prompt,
                                   restaurant_query_prompt,
                                   accommodation_query_prompt,
                                   project_context,
                                   answer_prompt,
                                   rewrite_question_prompt
                                   )

class State(TypedDict):
    question:                    Annotated[str, "Question"] # 질문
    query:                       Annotated[str, "Query"] # SQL Query문
    query_accommodation:         Annotated[str, "Query_Accommodation"]
    query_restaurant:            Annotated[str, "Query_Restaurant"]
    query_tourinfo:              Annotated[str, "Query_TourInfo"]
    fetch_db_accommodation:      Annotated[str, "Fetch_db_accommodation"]
    fetch_db_restaurant:         Annotated[str, "Fetch_db_restaurant"]
    fetch_db_tourinfo:           Annotated[str, "Fetch_db_tourinfo"]
    answer:                      Annotated[str, "Answer"] # 답변
    messages:                    Annotated[list, add_messages] # 메시지 (누적되는 list)
    relevance:                   Annotated[str, "Relevance"] # 관련성 여부
    # 병렬 처리 결과물 저장할 필드들 (reducer 사용)
    all_queries:                 Annotated[dict, operator.or_]
    all_results:                 Annotated[dict, operator.or_]

### LLM 정의
from langchain_google_genai                      import GoogleGenerativeAI
from langchain_core.prompts                      import PromptTemplate
from langchain.chains                            import create_sql_query_chain
from langchain_community.utilities               import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from dotenv                                      import load_dotenv, find_dotenv
# .env 파일에서 GOOGLE API 키를 불러와 환경변수에 설정
load_dotenv(find_dotenv())
import datetime

import os



# DB 엔진 설정 (환경에 맞게 수정)
HOST                 = os.getenv("DB_HOST")
PORT                 = os.getenv("DB_PORT")
USERNAME             = os.getenv("DB_USER")
PASSWORD             = os.getenv("DB_PASSWORD")
DB_SCHEMA            = os.getenv("DB_NAME")

mysql_uri = f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB_SCHEMA}"

os.environ["GOOGLE_API_KEY"] = os.getenv("KNY_GOOGLE_API_KEY")

llm = GoogleGenerativeAI(model='gemini-2.0-flash')

city = "서울"
district = "성동구"
travel_theme = "문화 체험"
companions = "연인"
group_size = 2
meal_schedule = {"breakfast": "home",
                "lunch": "out",
                "dinner": "out"}

# Day 단위로 하기 때문에 Day 바뀌면 바뀌지, 한 Day안에서 바뀔 일은 없음 (챗으로 변수가 생기면 나중에 추가해서 구현해야 함)
gu = '강남구' # SQL문 "강남구"를 포함해서 가져와
how_long = '2박3일'
travel_theme = '산여행'
eat_number = '아침/점심/저녁' #아침, 점심, 저녁 중복 선택 가능 # 문자열 # 식당을 몇 개로 넣을지를 알아야 하기 때문
# 브레이크 타임, 영업시간 고려해서 뽑아와야 함

with_who = '가족' #연인, 친구, 가족 # 문자열 -> # 물어보긴 하지만, LG에 넣지는 않음
party_size = 4 #여행인원 수 # int # 물어보긴 하지만, LG에 넣지는 않음
# 1안) 위 2개는 top 20개를 뽑아서 llm한테 추천해달라고 하는 형식

## 2안) 네이버 블로그 api 이용해서 '가족'이면 '가족'이랑 명소명/식당/숙소 같이 넣어서 검색한 후에 결과를 받아서
##      LLM이 추천하는 형식

## 날씨 데이터를 명소 단위로 불러오지 말고, 구 단위로 불러오자 (설득이 필요함)
### 구단위로 25개씩 받아오는 방법

location_preference = "popular" # "popluar":혼잡한지역 or "quiet":조용한지역 # 추가 데이터



# 현재 시점(YYYY-MM-DD HH:MM:SS) 저장

now_date = datetime.datetime.now().strftime("%Y-%m-%d")
now_time = datetime.datetime.now().strftime("%H:%M:%S")
now_date_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

db = SQLDatabase.from_uri(mysql_uri)

# SQL 실행 도구 정의
execute_query = QuerySQLDatabaseTool(db=db)

#################################################

# 2. 노드 정의
## def

def create_query_tourinfo(state: State) -> dict:
    
    question = state["question"]
    # 사용자의 요청에 맞는 쿼리 생성
    write_query = create_sql_query_chain(llm,
                                         db,
                                         tourinfo_query_prompt.partial(
                                             dialect=db.dialect,
                                             current_time=now_date_time,
                                             table_info=tourinfo_table_info,
                                             top_k=20,
                                             gu=district
                                             )
                                         )
    
    result = write_query.invoke({"question": question})
    
    clean_query = result.replace("```sql", "").replace("```", "").strip()
    
    # return State(query_tourinfo=clean_query)
    return {"query_tourinfo": clean_query,
            "all_queries": {"tourinfo": clean_query}}

def create_query_accommodation(state: State) -> dict:
    
    question = state["question"]
    # 사용자의 요청에 맞는 쿼리 생성
    write_query = create_sql_query_chain(llm,
                                         db,
                                         accommodation_query_prompt.partial(
                                             dialect=db.dialect,
                                             current_time=now_date_time,
                                             table_info=accommodation_table_info,
                                             top_k=20,
                                             gu=district
                                             )
                                         )
    
    result = write_query.invoke({"question": question})
    
    clean_query = result.replace("```sql", "").replace("```", "").strip()
    
    return {"query_accommodation": clean_query,
            "all_queries": {"accommodation": clean_query}}

def create_query_restaurant(state: State) -> dict:
    
    question = state["question"]
    # 사용자의 요청에 맞는 쿼리 생성
    write_query = create_sql_query_chain(llm,
                                         db,
                                         restaurant_query_prompt.partial(
                                             dialect=db.dialect,
                                             current_time=now_date_time,
                                             table_info=restaurant_table_info,
                                             top_k=20,
                                             gu=district
                                             )
                                         )
    
    result = write_query.invoke({"question": question})
    
    clean_query = result.replace("```sql", "").replace("```", "").strip()
    
    return {"query_restaurant": clean_query,
            "all_queries": {"restaurant": clean_query}}


def fetch_db_tourinfo(state: State) -> dict:
    
    query = state["query_tourinfo"]
    
    fetch_db = execute_query.invoke({"query": query})
    
    # return State(fetch_db_tourinfo=fetch_db)
    return {"fetch_db_tourinfo": fetch_db,
            "all_results": {"tourinfo": fetch_db}}

def fetch_db_accommodation(state: State) -> dict:
    
    query = state["query_accommodation"]
    
    fetch_db = execute_query.invoke({"query": query})
    
    return {"fetch_db_accommodation": fetch_db,
            "all_results": {"accommodation": fetch_db}}

def fetch_db_restaurant(state: State) -> dict:
    
    query = state["query_restaurant"]
    
    fetch_db = execute_query.invoke({"query": query})
    
    return {"fetch_db_restaurant": fetch_db,
            "all_results": {"restaurant": fetch_db}}


def generate_message(state: State) -> State:
    # gemini 답변
    question = state["question"]
    
    all_queries = state["all_queries"]
    all_results = state["all_results"]
    
    answer_chain = answer_prompt.partial(project_context=project_context,
                                         city=city,
                                         district=district,
                                         travel_theme=travel_theme,
                                         companions=companions,
                                         group_size=group_size,
                                         breakfast=meal_schedule['breakfast'],
                                         lunch=meal_schedule['lunch'],
                                         dinner=meal_schedule['dinner']
                                         ) | llm | StrOutputParser()

    answer = answer_chain.invoke({
        "question": question,
        "query_tourinfo": all_queries["tourinfo"],
        "query_accommodation": all_queries["accommodation"],
        "query_restaurant": all_queries["restaurant"],
        "fetch_db_tourinfo": all_results["tourinfo"],
        "fetch_db_accommodation": all_results["accommodation"],
        "fetch_db_restaurant": all_results["restaurant"]
    })
    
    return State(answer=answer)

from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# 3. 그래프 정의 및 엣지 연결

# Langgraph.graph에서 StateGraph와 END를 가져옵니다.
graph = StateGraph(State)

# 노드 추가

graph.add_node("create_query_tourinfo", create_query_tourinfo)
graph.add_node("create_query_accommodation", create_query_accommodation)
graph.add_node("create_query_restaurant", create_query_restaurant)
graph.add_node("fetch_tourinfo", fetch_db_tourinfo)
graph.add_node("fetch_accommodation", fetch_db_accommodation)
graph.add_node("fetch_restaurant", fetch_db_restaurant)

graph.add_node("generate_message", generate_message)

# 엣지로 노드 연결

graph.add_edge(START         , "create_query_tourinfo")
graph.add_edge(START         , "create_query_accommodation")
graph.add_edge(START         , "create_query_restaurant")
graph.add_edge("create_query_tourinfo", "fetch_tourinfo")
graph.add_edge("create_query_accommodation", "fetch_accommodation")
graph.add_edge("create_query_restaurant", "fetch_restaurant")

graph.add_edge(["fetch_tourinfo", "fetch_accommodation", "fetch_restaurant"], "generate_message")

graph.add_edge("generate_message", END)

# 기록을 위한 메모리 저장소 생성
memory = MemorySaver()

# 그래프 컴파일
app = graph.compile(checkpointer=memory)

from IPython.display import Image, display

# 그래프 시각화
# display(Image(app.get_graph().draw_mermaid_png()))

from langchain_core.runnables import RunnableConfig

config = RunnableConfig(recursion_limit=10,
                        configurable={"thread_id": "7"})

response = app.invoke({"question": "성동구"}, config=config)

print(response['answer'])

죄송합니다. 데이터베이스 연결 문제로 인해 성동구에 대한 식당, 숙소, 관광지 정보를 가져올 수 없습니다. 정보가 제한적이므로 완벽한 여행 계획을 제공하기 어렵습니다. 하지만, 일반적인 서울 성동구 산여행 코스를 연인과 함께 2명이서 즐기실 수 있도록 대략적인 추천을 해 드리겠습니다. 아침은 집에서 드시고, 점심과 저녁은 밖에서 드시는 것을 기준으로 합니다.

**<간략한 브리핑>** [아침 식사] -> [응봉산] -> [점심 식사] -> [서울숲] -> [저녁 식사] -> [숙소]

1. 아침식사
   **집**
   - 종류: 한식
   - 이유: 든든하게 하루를 시작하기 위해 집에서 편안하게 아침 식사를 즐기세요.
   - 이동: N/A

2. 관광지
   **응봉산**
   - 종류: 산
   - 이유: 성동구에서 접근성이 좋고, 정상에서 멋진 서울 야경을 감상할 수 있는 산입니다. 연인과 함께 가볍게 산책하기 좋습니다.
   - 이동: 집에서 응봉산까지 대중교통 이용 시 약 30분 예상됩니다. 자차 이용 시 약 15분 정도 소요됩니다.

3. 점심식사
   **성수동 맛집**
   - 종류: 다양한 종류의 식당 (파스타, 햄버거, 한식 등)
   - 이유: 성수동은 트렌디한 맛집이 많아 연인과 함께 즐기기에 좋습니다. 응봉산에서 성수동까지 이동하여 다양한 선택지 중에서 원하는 음식을 즐기세요.
   - 이동: 응봉산에서 성수동까지 대중교통 이용 시 약 20분, 자차 이용 시 약 10분 정도 소요됩니다.

4. 관광지
   **서울숲**
   - 종류: 공원
   - 이유: 넓은 잔디밭과 아름다운 산책로가 있어 연인과 함께 여유로운 시간을 보내기에 좋습니다. 사진 찍기 좋은 장소도 많습니다.
   - 이동: 성수동에서 서울숲까지 도보로 약 15분, 대중교통 이용 시 약 5분 정도 소요됩니다.

5. 저녁식사
   **서울숲 근처 맛집**
   - 종류: 다양한 종류의 식당 (이탈리안, 아시안 퓨전 등)
   - 이유: 서울숲 근처에는 분위기 좋은 레스토